In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __init__ import *

PROJECT_JSON = kraft.read_json("../project.json")

PATH = make_path_dict(PROJECT_JSON)

In [ ]:
signal = pd.read_csv(PATH["feature_x_sample.signal.tsv"], sep="\t", index_col=0)

signal.columns.name = PROJECT_JSON["sample_alias"]

signal.fillna(value=0, inplace=True)

In [ ]:
k_mfcc = kraft.mf_consensus_cluster_dataframe_with_ks(
    signal,
    PROJECT_JSON["mf_ks"],
    n_job=PROJECT_JSON["n_job"],
    n_clustering=PROJECT_JSON["mf_n_clustering"],
    plot_dataframe=signal.size < PROJECT_JSON["plot_heat_map_max_size"],
    plot_w=signal.shape[0] < PROJECT_JSON["plot_cluster_max_size"],
    plot_h=signal.shape[1] < PROJECT_JSON["plot_cluster_max_size"],
    directory_path=PATH["mf/"],
)

In [ ]:
for k in PROJECT_JSON["mf_ks"]:

    for w_or_h, axis in (("w", 0), ("h", 1)):

        dataframe = pd.read_csv(
            os.path.join(PATH["mf/"], str(k), "{}.tsv".format(w_or_h)),
            sep="\t",
            index_col=0,
        )

        if w_or_h == "w":

            dataframe.columns.name = "Factor"

        kraft.apply_function_on_slices_from_2_dataframes(
            dataframe,
            dataframe,
            axis,
            kraft.compute_information_coefficient_between_2_vectors,
            layout={"title": {"text": "MF K={} {}".format(k, w_or_h.title())}},
        )